# Titanic - Machine Learning from Disaster

<img src="https://www.cnnbrasil.com.br/wp-content/uploads/sites/12/2023/06/titanic_navio_1912-e1687528104691.jpg?w=876&h=484&crop=1" width=900>

- Vamos empregar os dados atualmente acessíveis no Kaggle. [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/titanic)
    - Trata-se de um conjunto de dados destinado a uma competição
    - A avaliação do desempenho decorre com base na acurácia:

  *Sua pontuação reflete a porcentagem de passageiros que você é capaz de prever com precisão. Esse índice é conhecido como acurácia.*

### Começando a entender a base

- Utilizaremos o ydata-profiling (antigo Pandas Profiling) para começar a entender a base:
    - https://github.com/ydataai/ydata-profiling

In [ ]:
# Importando o pandas
import pandas as pd

In [ ]:
# Visualizando a base de treino
train = pd.read_csv('train.csv')
train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
#Instalando o  ydata-profiling
!pip install ydata-profiling

In [ ]:
# Importando o ProfileReport do ydata-profiling
from ydata_profiling import ProfileReport

In [ ]:
# Gerando a visualização
profile = ProfileReport(train, title="titanic_train")

In [ ]:
# Criando o relatório
profile.to_file("titanic_train.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

- **Analisando a base de dados:**

In [ ]:
# Verificando as informações da base de treino
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
# Verificando o tipo dos dados de treino
train.dtypes.value_counts()

int64      5
object     5
float64    2
dtype: int64

In [ ]:
# Identificando os valores nulos
train.isnull().sum().sort_values(ascending=False).head()

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

- **Visualizando a base de teste**

In [ ]:
# Visualizando a base de teste
test = pd.read_csv('test.csv')
test.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
# Verificando as informações da base de teste
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [ ]:
# Identificando os valores nulos
test.isnull().sum().sort_values(ascending=False).head()

Cabin          327
Age             86
Fare             1
PassengerId      0
Pclass           0
dtype: int64

- Identificamos colunas na base de teste com valores ausentes que estão completos na base de treino (nesse cenário, é necessário tratar essas colunas exclusivamente na base de teste).

### Realizando os tratamentos iniciais dos dados

- **Tratando valores nulos**

In [ ]:
# Verificando novamente valores vazios para a base de treino
train.isnull().sum().sort_values(ascending=False).head()

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

In [ ]:
# Retornando o shape da base
train.shape

(891, 12)

In [ ]:
# Visualizando novamente a base
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [ ]:
# Verificando a cardinalidade dos dados
train.nunique().sort_values(ascending=False)

PassengerId    891
Name           891
Ticket         681
Fare           248
Cabin          147
Age             88
SibSp            7
Parch            7
Pclass           3
Embarked         3
Survived         2
Sex              2
dtype: int64

- Colunas que exibem alta cardinalidade, como aquelas relacionadas a nomes, números de bilhetes e até mesmo códigos de cabine, tendem a não contribuir positivamente para o modelo, pois podem reduzir sua capacidade de generalização. Assim sendo, optaremos por remover essas colunas.

In [ ]:
# Eliminando para a base de treino
train = train.drop(['Name', 'Ticket','Cabin'], axis=1)

In [ ]:
# Também eliminando para a base de teste
test = test.drop(['Name', 'Ticket','Cabin'], axis=1)

Chegou o momento de abordar a coluna "**Age**", a qual apresenta lacunas tanto nos dados de treino quanto nos de teste.

 Dado que essa coluna contém as idades dos passageiros, uma abordagem viável seria empregar a média das idades para preencher os espaços vazios.


In [ ]:
# Verificando a média das idades para a base de treino
train.Age.mean()

In [ ]:
# Podemos agora selecionar as colunas com idade vazia e substituir por essa média
train.loc[train.Age.isnull(),'Age'] = train.Age.mean()

In [ ]:
# E fazendo o mesmo para a base de teste
test.loc[test.Age.isnull(),'Age'] = test.Age.mean()

- Para a base de treino, ainda temos 2 valores vazios na coluna **Embarked**
  
  Podemos verificar todos os valores nessa coluna e substituir os valores vazios pela **moda**

In [ ]:
# Verificando os valores na coluna Embarked
train.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [ ]:
# Selecionando a moda dessa coluna
train.Embarked.mode()[0]

'S'

In [ ]:
# Substituindo os valores vazios
train.loc[train.Embarked.isnull(),'Embarked'] = train.Embarked.mode()[0]

Para concluir, na base de teste, ainda identificamos a presença de 1 valor ausente na coluna "**Fare**". Podemos aplicar a mesma estratégia que utilizamos para a coluna "Age" e preencher esse valor com a moda.

In [ ]:
# Verificando a média da coluna Fare para a base de teste
test.loc[test.Fare.isnull(),'Fare'] = test.Fare.mean()

- **Verificando novamente os valores vazios**

In [ ]:
# Para a base de treino
train.isnull().sum().sort_values(ascending=False)

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [ ]:
# Para a base de teste
test.isnull().sum().sort_values(ascending=False)

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

### Considerando apenas as colunas que não são de texto
Na primeira parte da Análise do Titanic, nosso foco estará exclusivamente na manipulação das colunas numéricas, deixando de lado aquelas que contêm informações em formato de texto.

In [ ]:
# Verificando as colunas que não são texto na base de treino
col_train_numeric=train.columns[train.dtypes != 'object']

In [ ]:
# Selecionando apenas os valores numéricos da base de treino
train_numeric=train.loc[:,col_train_numeric]
train_numeric

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.000000,1,0,7.2500
1,2,1,1,38.000000,1,0,71.2833
2,3,1,3,26.000000,0,0,7.9250
3,4,1,1,35.000000,1,0,53.1000
4,5,0,3,35.000000,0,0,8.0500
...,...,...,...,...,...,...,...
886,887,0,2,27.000000,0,0,13.0000
887,888,1,1,19.000000,0,0,30.0000
888,889,0,3,29.699118,1,2,23.4500
889,890,1,1,26.000000,0,0,30.0000


In [ ]:
# E para a base de teste
col_test_numeric=test.columns[test.dtypes != 'object']

In [ ]:
# e os valores numéricos da base de teste
test_numeric=test.loc[:,col_test_numeric]
test_numeric

,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,892,3,34.50000,0,0,7.8292
1,893,3,47.00000,1,0,7.0000
2,894,2,62.00000,0,0,9.6875
3,895,3,27.00000,0,0,8.6625
4,896,3,22.00000,1,1,12.2875
...,...,...,...,...,...,...
413,1305,3,30.27259,0,0,8.0500
414,1306,1,39.00000,0,0,108.9000
415,1307,3,38.50000,0,0,7.2500
416,1308,3,30.27259,0,0,8.0500


### Podemos agora selecionar um modelo para classificar esses dados
- Para começar, vamos testar entre:
    - **Árvore de classificação**
        - https://scikit-learn.org/stable/modules/tree.html#classification
    - **Classificação dos vizinhos mais próximos**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

- Antes de usar os algoritmos, precisamos separar a base de treino em **treino e validação**
    - Vamos fazer isso utilizando o **train_test_split**
        - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
# Separando a base de treino em X e y
x = train_numeric.drop(['PassengerId','Survived'], axis=1)
y= train.Survived

In [ ]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.33, random_state=42)

- Para a **árvore de classificação**

In [ ]:
# Fazendo a importação
from sklearn import tree

In [ ]:
# Criando o classificador
clf_ac = tree.DecisionTreeClassifier()

In [ ]:
# Fazendo o fit com os dados
clf_ac = clf_ac.fit(X_train, y_train)

In [ ]:
# Fazendo a previsão
y_pred_ac = clf_ac.predict(X_val)

- Para o **KNeighborsClassifier**

In [ ]:
# Importando
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Criando o classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# Fazendo o fit com os dados
clf_knn = clf_knn.fit(X_train, y_train)

In [ ]:
# Fazendo a previsão
y_pred_knn = clf_knn.predict(X_val)

- E para a **Regressão Logística**

In [ ]:
# Importando
from sklearn.linear_model import LogisticRegression

In [ ]:
# Criando o classificador
clf_rl = LogisticRegression(random_state=0).fit(x, y)

In [ ]:
# Fazendo o fit com os dados
clf_rl = clf_rl.fit(X_train, y_train)

In [ ]:
# Fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)

### E, para finalizar, vamos avaliar esses modelos
- Para fazer essa análise, podemos usar:
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

- Avaliando a **acurácia**

In [ ]:
# Importando
from sklearn.metrics import accuracy_score

In [ ]:
# Para a árvore
accuracy_score(y_val, y_pred_ac)

0.6271186440677966

In [ ]:
# Para o knn
accuracy_score(y_val, y_pred_knn)

0.6610169491525424

In [ ]:
# Para a regressão logística
accuracy_score(y_val, y_pred_rl)

0.7254237288135593

- Avaliando a **matriz de confusão**

In [ ]:
# Importando
from sklearn.metrics import confusion_matrix

In [ ]:
# Para a árvore
confusion_matrix(y_val, y_pred_ac)

array([[129,  46],
       [ 64,  56]])

In [ ]:
# Para o knn
confusion_matrix(y_val, y_pred_knn)

array([[133,  42],
       [ 58,  62]])

In [ ]:
# Para a regressão logística
confusion_matrix(y_val, y_pred_rl)

array([[156,  19],
       [ 62,  58]])

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

In [ ]:
# Visualizando o X_train
X_train.head(3)

,Pclass,Age,SibSp,Parch,Fare
6,1,54.000000,0,0,51.8625
718,3,29.699118,0,0,15.5000
685,2,25.000000,1,2,41.5792


In [ ]:
# Visualizando a base de teste (apenas com valores numéricos)
test_numeric.head(3)

,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,892,3,34.5,0,0,7.8292
1,893,3,47.0,1,0,7.0000
2,894,2,62.0,0,0,9.6875


In [ ]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id
x_test = test_numeric.drop('PassengerId',axis=1)

In [ ]:
# Utilizando a regressão logística na base de teste
y_pred = clf_rl.predict(x_test)

In [ ]:
# Criando uma nova coluna com a previsão na base de teste
test['Survived'] = y_pred

In [ ]:
# Selecionando apenas a coluna de Id e Survived para fazer o envio
base_envio = test[['PassengerId','Survived']]

In [ ]:
# Exportando para um csv
base_envio.to_csv('resultados.csv', index=False)